# baseline experiment

In [ ]:
!pip install -q -U diffusers transformers huggingface_hub accelerate sentencepiece

In [ ]:
from google.colab import drive
import os

drive.mount("/content/drive")

ROOT = "/content/drive/MyDrive/thesis2"


HF_CACHE_DIR = f"{ROOT}/sd35_cache"
os.makedirs(HF_CACHE_DIR, exist_ok=True)

# Phase 1: smile experiment folders
PHASE_ROOT = f"{ROOT}/phase1_smile"
IMG_ROOT = f"{PHASE_ROOT}/images"
LOG_ROOT = f"{PHASE_ROOT}/logs"

os.makedirs(IMG_ROOT, exist_ok=True)
os.makedirs(LOG_ROOT, exist_ok=True)

os.environ["HF_HOME"] = HF_CACHE_DIR
os.environ["HF_HUB_CACHE"] = HF_CACHE_DIR

print("HF cache:", HF_CACHE_DIR)
print("Phase 1 images:", IMG_ROOT)
print("Phase 1 logs:", LOG_ROOT)


In [ ]:
!pip install -q -U diffusers transformers huggingface_hub accelerate sentencepiece

from huggingface_hub import login
from diffusers import StableDiffusion3Pipeline
import torch



model_id = "stabilityai/stable-diffusion-3.5-medium"

pipe = StableDiffusion3Pipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    token=HF_TOKEN,
)

pipe = pipe.to("cuda")
pipe.enable_attention_slicing()

print("Loaded on:", torch.cuda.get_device_name(0))


In [ ]:
# === Prompts for Phase 1: neutral / smile ===
BASE_FACE_PROMPT = (
    "a photorealistic portrait of a human face, studio lighting, "
    "high resolution, natural skin texture, realistic anatomy, "
    "professional photography, symmetric face, looking forward"
)

CONDITIONS = {
    "neutral": {
        "prompt": BASE_FACE_PROMPT
        + ", neutral expression, no smile, relaxed mouth, closed lips",
    },
    "smiling": {
        "prompt": BASE_FACE_PROMPT
        + ", smiling, visible teeth, joyful expression, warm smile",
    }
}


N_VARIANTS = 30       # images per condition
BASE_SEED = 123
HEIGHT = 768
WIDTH = 768
NUM_STEPS = 30
GUIDANCE_SCALE = 7.0


In [ ]:
import torch
import csv
from datetime import datetime

def get_condition_paths(condition_name: str):
    cond_img_dir = os.path.join(IMG_ROOT, condition_name)
    cond_log_path = os.path.join(LOG_ROOT, f"seed_diversity_{condition_name}.csv")
    os.makedirs(cond_img_dir, exist_ok=True)
    return cond_img_dir, cond_log_path

def generate_seed_diversity_for_condition(
    condition_name: str,
    n_variants: int = N_VARIANTS,
    base_seed: int = BASE_SEED,
    num_inference_steps: int = NUM_STEPS,
    guidance_scale: float = GUIDANCE_SCALE,
    height: int = HEIGHT,
    width: int = WIDTH,
):
    assert condition_name in CONDITIONS, f"Unknown condition: {condition_name}"
    prompt = CONDITIONS[condition_name]["prompt"]

    cond_img_dir, cond_log_path = get_condition_paths(condition_name)

    # init CSV if needed
    if not os.path.exists(cond_log_path):
        with open(cond_log_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                "timestamp",
                "condition",
                "prompt",
                "seed",
                "num_inference_steps",
                "guidance_scale",
                "height",
                "width",
                "image_path",
            ])

    for i in range(n_variants):
        seed = base_seed + i
        generator = torch.Generator(device="cuda").manual_seed(seed)

        print(f"[{condition_name}] {i+1}/{n_variants} (seed={seed})")
        image = pipe(
            prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            height=height,
            width=width,
            generator=generator,
        ).images[0]

        fname = f"{condition_name}_seed{seed}.png"
        save_path = os.path.join(cond_img_dir, fname)
        image.save(save_path)

        with open(cond_log_path, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                datetime.now().isoformat(timespec="seconds"),
                condition_name,
                prompt,
                seed,
                num_inference_steps,
                guidance_scale,
                height,
                width,
                save_path,
            ])

        print("  saved:", save_path)

    print(f" Done for '{condition_name}'. Log at {cond_log_path}")


In [ ]:
generate_seed_diversity_for_condition("neutral")
generate_seed_diversity_for_condition("smiling")


In [ ]:
!pip install -q lpips scikit-image pandas matplotlib seaborn


In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from skimage.metrics import structural_similarity as ssim
import lpips

device = "cuda" if torch.cuda.is_available() else "cpu"
lpips_model = lpips.LPIPS(net='vgg').to(device)
lpips_model.eval()

def load_image(path, size=None):
    img = Image.open(path).convert("RGB")
    if size is not None:
        img = img.resize(size, Image.BICUBIC)
    return img

def img_to_numpy(img):
    return np.asarray(img).astype(np.float32) / 255.0

def img_to_lpips_tensor(img):
    arr = np.asarray(img).astype(np.float32) / 255.0
    arr = (arr * 2.0) - 1.0
    arr = torch.from_numpy(arr).permute(2, 0, 1).unsqueeze(0)
    return arr.to(device)

def compute_pairwise_metrics_for_condition(condition_name: str):
    cond_img_dir, cond_log_path = get_condition_paths(condition_name)
    print(f"\n=== Metrics for: {condition_name} ===")
    print("Images:", cond_img_dir)
    print("Log:", cond_log_path)

    df = pd.read_csv(cond_log_path)
    df_cond = df[df["condition"] == condition_name].reset_index(drop=True)
    print("Rows:", len(df_cond))

    images_np, images_lpips, paths_used = [], [], []
    target_size = None

    for _, row in df_cond.iterrows():
        path = row["image_path"]
        if not os.path.exists(path):
            print(" missing:", path)
            continue

        if target_size is None:
            target_size = Image.open(path).convert("RGB").size

        img = load_image(path, size=target_size)
        images_np.append(img_to_numpy(img))
        images_lpips.append(img_to_lpips_tensor(img))
        paths_used.append(path)

    print("Loaded", len(images_np), "images of size", target_size)

    results = []
    n = len(images_np)

    for i in range(n):
        for j in range(i+1, n):
            seed_i = int(df_cond.loc[i, "seed"])
            seed_j = int(df_cond.loc[j, "seed"])

            arr_i, arr_j = images_np[i], images_np[j]

            mse_val = float(np.mean((arr_i - arr_j)**2))
            ssim_val = float(ssim(arr_i, arr_j, channel_axis=-1, data_range=1.0))
            with torch.no_grad():
                lpips_val = float(lpips_model(images_lpips[i], images_lpips[j]).item())

            results.append({
                "condition": condition_name,
                "seed_i": seed_i,
                "seed_j": seed_j,
                "image_i": paths_used[i],
                "image_j": paths_used[j],
                "MSE": mse_val,
                "SSIM": ssim_val,
                "LPIPS": lpips_val,
            })

            print(f"{condition_name} ({seed_i},{seed_j}) -> "
                  f"MSE={mse_val:.5f}, SSIM={ssim_val:.4f}, LPIPS={lpips_val:.4f}")

    metrics_df = pd.DataFrame(results)
    pairwise_path = os.path.join(LOG_ROOT, f"pairwise_{condition_name}.csv")
    metrics_df.to_csv(pairwise_path, index=False)
    print("saved pairwise to:", pairwise_path)

    # summary
    summary = {
        "condition": condition_name,
        "N_pairs": len(metrics_df),
        "LPIPS_mean": metrics_df["LPIPS"].mean(),
        "LPIPS_std":  metrics_df["LPIPS"].std(),
        "SSIM_mean":  metrics_df["SSIM"].mean(),
        "SSIM_std":   metrics_df["SSIM"].std(),
        "MSE_mean":   metrics_df["MSE"].mean(),
        "MSE_std":    metrics_df["MSE"].std(),
    }
    summary_df = pd.DataFrame([summary])
    display(summary_df)

    summary_path = os.path.join(LOG_ROOT, "seed_diversity_summary_smile.csv")
    if os.path.exists(summary_path):
        old = pd.read_csv(summary_path)
        pd.concat([old, summary_df], ignore_index=True).to_csv(summary_path, index=False)
    else:
        summary_df.to_csv(summary_path, index=False)

    print("updated summary at:", summary_path)
    return metrics_df, summary_df


In [ ]:
metrics_neutral, summary_neutral = compute_pairwise_metrics_for_condition("neutral")
metrics_smiling, summary_smiling = compute_pairwise_metrics_for_condition("smiling")


# guidance

In [ ]:
import os


try:
    ROOT
except NameError:
    ROOT = "/content/drive/MyDrive/thesis2"

P2_ROOT = f"{ROOT}/phase2_guidance_faces"
P2_RESULTS_ROOT = f"{P2_ROOT}/results/faces"
P2_LOG_ROOT = f"{P2_ROOT}/logs"

os.makedirs(P2_RESULTS_ROOT, exist_ok=True)
os.makedirs(P2_LOG_ROOT, exist_ok=True)

print("Phase 2 root:", P2_ROOT)
print("Results root:", P2_RESULTS_ROOT)
print("Logs root:", P2_LOG_ROOT)

# Prompts for Phase 2
P2_PROMPTS = {
    "neutral": (
          "a photorealistic portrait of a human face, studio lighting, "
          "high resolution, natural skin texture, realistic anatomy, "
          "professional photography, symmetric face, looking forward, "
          "neutral expression, no smile, relaxed mouth, closed lips"
),
    "smiling": (
        "a photorealistic portrait of a human face, studio lighting, "
        "high resolution, natural skin texture, realistic anatomy, "
        "professional photography, symmetric face, looking forward, "
        "smiling, visible teeth, joyful expression, warm smile"
    )
}

P2_GUIDANCE_LEVELS = [1.0, 3.0, 5.0, 7.5, 10.0]
P2_N_VARIANTS = 20
P2_BASE_SEED = 12345
P2_HEIGHT = 768
P2_WIDTH = 768
P2_NUM_STEPS = 18


In [ ]:
import torch
import csv
from datetime import datetime

def p2_get_paths(condition: str, guidance: float):
    """Return (img_dir, log_path) for this condition + guidance."""
    g_str = str(guidance).replace('.', '_')
    img_dir = os.path.join(P2_RESULTS_ROOT, condition, f"guidance_{g_str}")
    os.makedirs(img_dir, exist_ok=True)

    # One log per condition across all guidance
    log_path = os.path.join(P2_LOG_ROOT, f"seed_diversity_{condition}_guidance.csv")
    return img_dir, log_path

def p2_generate_for_condition_guidance(
    condition: str,
    guidance: float,
    n_variants: int = P2_N_VARIANTS,
    base_seed: int = P2_BASE_SEED,
    num_inference_steps: int = P2_NUM_STEPS,
    height: int = P2_HEIGHT,
    width: int = P2_WIDTH,
):
    assert condition in P2_PROMPTS, f"Unknown condition: {condition}"
    prompt = P2_PROMPTS[condition]

    img_dir, log_path = p2_get_paths(condition, guidance)

    # Create CSV header if needed
    if not os.path.exists(log_path):
        with open(log_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                "timestamp",
                "condition",
                "prompt",
                "guidance_scale",
                "seed",
                "num_inference_steps",
                "height",
                "width",
                "image_path",
            ])

    for i in range(n_variants):
        seed = base_seed + i
        gen = torch.Generator(device="cuda").manual_seed(seed)

        print(f"[{condition}] guidance={guidance} | {i+1}/{n_variants} (seed={seed})")
        image = pipe(
            prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance,
            height=height,
            width=width,
            generator=gen,
        ).images[0]

        g_str = str(guidance).replace('.', '_')
        fname = f"{condition}_g{g_str}_seed{seed}.png"
        save_path = os.path.join(img_dir, fname)
        image.save(save_path)

        with open(log_path, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                datetime.now().isoformat(timespec="seconds"),
                condition,
                prompt,
                guidance,
                seed,
                num_inference_steps,
                height,
                width,
                save_path,
            ])

        print("saved:", save_path)

    print(f"Done for condition={condition}, guidance={guidance}")


In [ ]:
for cond in ["neutral", "smiling"]:
    for g in P2_GUIDANCE_LEVELS:
        p2_generate_for_condition_guidance(cond, g)


In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from skimage.metrics import structural_similarity as ssim
import lpips


try:
    lpips_model
except NameError:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    lpips_model = lpips.LPIPS(net='vgg').to(device)
    lpips_model.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"

def p2_load_image(path, size=None):
    img = Image.open(path).convert("RGB")
    if size is not None:
        img = img.resize(size, Image.BICUBIC)
    return img

def p2_img_to_numpy(img):
    return np.asarray(img).astype(np.float32) / 255.0

def p2_img_to_lpips_tensor(img):
    arr = np.asarray(img).astype(np.float32) / 255.0
    arr = (arr * 2.0) - 1.0
    arr = torch.from_numpy(arr).permute(2, 0, 1).unsqueeze(0)
    return arr.to(device)

def p2_compute_pairwise_for(condition: str, guidance: float):
    img_dir, log_path = p2_get_paths(condition, guidance)
    print(f"\n=== {condition} | guidance={guidance} ===")
    print("Log:", log_path)

    df = pd.read_csv(log_path)
    df_sub = df[(df["condition"] == condition) & (df["guidance_scale"] == guidance)].reset_index(drop=True)
    print("Rows:", len(df_sub))

    if len(df_sub) < 2:
        print("Not enough images for pairwise metrics.")
        return None, None

    images_np, images_lpips, paths_used = [], [], []
    target_size = None

    for _, row in df_sub.iterrows():
        path = row["image_path"]
        if not os.path.exists(path):
            print("missing:", path)
            continue

        if target_size is None:
            target_size = Image.open(path).convert("RGB").size

        img = p2_load_image(path, size=target_size)
        images_np.append(p2_img_to_numpy(img))
        images_lpips.append(p2_img_to_lpips_tensor(img))
        paths_used.append(path)

    print("Loaded", len(images_np), "images, size:", target_size)

    results = []
    n = len(images_np)

    for i in range(n):
        for j in range(i + 1, n):
            seed_i = int(df_sub.loc[i, "seed"])
            seed_j = int(df_sub.loc[j, "seed"])

            arr_i, arr_j = images_np[i], images_np[j]

            mse_val = float(np.mean((arr_i - arr_j) ** 2))
            ssim_val = float(ssim(arr_i, arr_j, channel_axis=-1, data_range=1.0))

            with torch.no_grad():
                lpips_val = float(lpips_model(images_lpips[i], images_lpips[j]).item())

            results.append({
                "condition": condition,
                "guidance_scale": guidance,
                "seed_i": seed_i,
                "seed_j": seed_j,
                "image_i": paths_used[i],
                "image_j": paths_used[j],
                "MSE": mse_val,
                "SSIM": ssim_val,
                "LPIPS": lpips_val,
            })

    metrics_df = pd.DataFrame(results)

    g_str = str(guidance).replace('.', '_')
    pairwise_name = f"seed_diversity_pairwise_{condition}_guidance_{g_str}.csv"
    pairwise_path = os.path.join(P2_LOG_ROOT, pairwise_name)
    metrics_df.to_csv(pairwise_path, index=False)
    print(" Saved pairwise metrics to:", pairwise_path)

    summary = {
        "condition": condition,
        "guidance_scale": guidance,
        "N_pairs": len(metrics_df),
        "LPIPS_mean": metrics_df["LPIPS"].mean(),
        "LPIPS_std":  metrics_df["LPIPS"].std(),
        "SSIM_mean":  metrics_df["SSIM"].mean(),
        "SSIM_std":   metrics_df["SSIM"].std(),
        "MSE_mean":   metrics_df["MSE"].mean(),
        "MSE_std":    metrics_df["MSE"].std(),
    }
    summary_df = pd.DataFrame([summary])
    display(summary_df)

    return metrics_df, summary_df


In [ ]:
def p2_run_all_metrics_for_condition(condition: str):
    all_summary = []
    for g in P2_GUIDANCE_LEVELS:
        metrics_df, summary_df = p2_compute_pairwise_for(condition, g)
        if summary_df is not None:
            all_summary.append(summary_df)

    if not all_summary:
        print("No metric summaries for", condition)
        return None

    summary_all = pd.concat(all_summary, ignore_index=True)
    summary_name = f"seed_diversity_summary_{condition}_guidance.csv"
    summary_path = os.path.join(P2_LOG_ROOT, summary_name)
    summary_all.to_csv(summary_path, index=False)
    print(f"\nSaved summary for {condition} to:", summary_path)
    display(summary_all)
    return summary_all

summary_neutral_p2 = p2_run_all_metrics_for_condition("neutral")
summary_smiling_p2 = p2_run_all_metrics_for_condition("smiling")


In [ ]:
def p2_metric_table(summary_df, metric: str, condition: str):
    tbl = summary_df[["guidance_scale", f"{metric}_mean"]].copy()
    tbl = tbl.sort_values("guidance_scale").reset_index(drop=True)
    tbl.columns = ["guidance_scale", f"{condition}_{metric}_mean"]
    return tbl

lpips_neutral_tbl = p2_metric_table(summary_neutral_p2, "LPIPS", "neutral")
lpips_smiling_tbl = p2_metric_table(summary_smiling_p2, "LPIPS", "smiling")

ssim_neutral_tbl = p2_metric_table(summary_neutral_p2, "SSIM", "neutral")
ssim_smiling_tbl = p2_metric_table(summary_smiling_p2, "SSIM", "smiling")

mse_neutral_tbl = p2_metric_table(summary_neutral_p2, "MSE", "neutral")
mse_smiling_tbl = p2_metric_table(summary_smiling_p2, "MSE", "smiling")

print("LPIPS vs guidance:")
display(lpips_neutral_tbl.merge(lpips_smiling_tbl, on="guidance_scale"))

print("SSIM vs guidance:")
display(ssim_neutral_tbl.merge(ssim_smiling_tbl, on="guidance_scale"))

print("MSE vs guidance:")
display(mse_neutral_tbl.merge(mse_smiling_tbl, on="guidance_scale"))


In [ ]:
import os
import re
import pandas as pd


GUIDANCE_LOG_DIR = "/content/drive/MyDrive/thesis2/phase2_guidance_faces/logs"

# 2) Regex to catch condition + guidance value from filenames like:
#     seed_diversity_pairwise_neutral_guidance_1_0.csv
pairwise_pattern = re.compile(
    r"seed_diversity_pairwise_(neutral|smiling)_guidance_([0-9_]+)\.csv"
)

rows = []

for fname in os.listdir(GUIDANCE_LOG_DIR):
    m = pairwise_pattern.match(fname)
    if not m:
        continue

    condition = m.group(1)                 # "neutral" or "smiling"
    g_str = m.group(2)                     # e.g. "1_0", "7_5", "10_0"
    guidance = float(g_str.replace("_", "."))

    fpath = os.path.join(GUIDANCE_LOG_DIR, fname)
    df = pd.read_csv(fpath)

    # make column names easier to handle
    df.columns = [c.lower() for c in df.columns]

    # figure out which metrics are present
    metric_names = []
    for cand in ["lpips", "ssim", "mse", "clip_cosine_dst", "clip_dist"]:
        if cand in df.columns:
            metric_names.append(cand)

    stats = {"condition": condition, "guidance": guidance}

    for mname in metric_names:
        stats[f"{mname}_mean"] = df[mname].mean()
        stats[f"{mname}_std"] = df[mname].std()

    rows.append(stats)

# 3) Put everything in a DataFrame
guidance_summary = pd.DataFrame(rows)

# nice ordering
guidance_summary = guidance_summary.sort_values(
    by=["condition", "guidance"]
).reset_index(drop=True)

guidance_summary
